In [1]:
import datetime
import json
import asyncio

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jakejongejans_0000313927
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [2]:
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();

In [4]:
def get_ip(node):
    interface = node.get_interface(network_name=f"Network-{node.get_site()}")
    return interface.get_ip_addr()

ips = [get_ip(node) for node in nodes];

print(ips);

['10.145.3.2', '10.145.3.3', '10.145.3.4', '10.137.8.2', '10.146.8.2']


In [5]:
inventory = (
    f"[kube_control_plane]\n"
    f"control ansible_host={ips[0]} ip={ips[0]} etcd_member_name=etcd1\n"
    f"\n"
    f"[etcd:children]\n"
    f"kube_control_plane\n"
    f"\n"
    f"[kube_node]\n"
    f"dynamos ansible_host={ips[1]} ip={ips[1]}\n"
)

for i, ip in enumerate(ips[2:]):
    inventory += f"worker-{i + 1} ansible_host={ip} ip={ip}\n"

with open('kubespray/inventory.ini', 'w') as f:
    f.write(inventory)

In [6]:
nodes[0].upload_file(local_file_path="node_scripts/control_kubespray_setup.sh", remote_file_path="kubespray_setup.sh");
nodes[0].execute("chmod +x kubespray_setup.sh && ./kubespray_setup.sh");

nodes[0].upload_file(local_file_path="kubespray/inventory.ini", remote_file_path="kubespray/inventory/dynamos/inventory.ini");
nodes[0].upload_file(local_file_path="kubespray/ansible.cfg", remote_file_path="kubespray/ansible.cfg");
nodes[0].upload_file(local_file_path="node_scripts/dot_kube.sh", remote_file_path="dot_kube.sh");
nodes[0].execute("chmod +x ./dot_kube.sh");
nodes[0].upload_file(local_file_path="/home/fabric/work/fabric_config/slice_key", remote_file_path="/home/ubuntu/.ssh/slice_key");
nodes[0].execute("chmod 600 /home/ubuntu/.ssh/slice_key");

branch 'release-2.27' set up to track 'origin/release-2.27'.
 Cloning into 'kubespray'...
Switched to a new branch 'release-2.27'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 143.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.4/479.4 kB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 38.9 MB/s eta 0:00:00
ansible [core 2.16.14]
  config file = /home/ubuntu/kubespray/ansible.cfg
  configured module search path = ['/home/ubuntu/kubespray/library']
  ansible p